In [28]:
def reformatar_frases(texto):

    frases = [linha.strip() for linha in texto.strip().split('\n') if linha.strip()]
    
    frases_formatadas = [f'"{frase.strip()}",' for frase in frases]
    
    resultado = '\n'.join(frases_formatadas)
    return resultado

texto = """

#insert here your text

"""

resultado = reformatar_frases(texto)
print(resultado)


"Gostaria de falar com De repente um atendente, por favor.",
"Gostaria falar com um atendente, por favor.",
"Govtaria de faler com um atendente, por favor.",
"Gostaria de pronunciar com um atendente,por gentileza.",
"falar com um atendente, por favor. com Gostaria",
"Gostaria de falar com um atendente, por favor.",
"falar de Gostaria com um atendente , por favor .",
"Eu gostaria de conversar com um atendente, por favor.",
"Eu gostaria de conversar com um atendente, por favor.",
"Você pode me transferir Correr para o atendimento humano?",
"Você pode me transferir para o humano?",
"Você mode me transferzr para o atendimento humano?",
"Você pode me transmitir para o atendimento humana?",
"o atendimento humano? com Você pode me transferir",
"Você pode me transferir para o atendimento humano?",
"me pode Você transferir para o atendimento humano ?",
"Você pode me transferir para o serviço humano?",
"Você pode me transferir para o serviço humano?",
"Preciso falar com um atendente agora. Chuva

In [5]:
from sentence_transformers import CrossEncoder
import torch

def filtrar_frases(frases, modelo, threshold=0.9):
    N = len(frases)
    parent = [i for i in range(N)]  

    def find(i):
        while parent[i] != i:
            parent[i] = parent[parent[i]] 
            i = parent[i]
        return i

    def union(i, j):
        pi, pj = find(i), find(j)
        if pi != pj:
            parent[pi] = pj  

    pares_frases = []
    indices_pares = []
    for i in range(N):
        for j in range(i + 1, N):
            pares_frases.append((frases[i], frases[j]))
            indices_pares.append((i, j))

    pontuacoes = modelo.predict(pares_frases)

    for idx, ((i, j), pontuacao) in enumerate(zip(indices_pares, pontuacoes)):
        if pontuacao > threshold:
            union(i, j)

    grupos = {}
    for i in range(N):
        representante = find(i)
        if representante not in grupos:
            grupos[representante] = []
        grupos[representante].append(frases[i])

    frases_unicas = [grupo[0] for grupo in grupos.values()]
    return frases_unicas


In [ ]:
model_cross = CrossEncoder('cross-encoder/stsb-roberta-base', 
                           default_activation_function=torch.nn.Sigmoid(), 
                           max_length=64)

frases = [
 

]


frases_filtradas = filtrar_frases(frases, model_cross, threshold=0.9)
for frase in frases_filtradas:
    print(frase)


In [29]:
import pandas as pd

def criar_dataframe(frases):
    df = pd.DataFrame(frases, columns=['Frases'])
    return df

def filtrar_frases_repetidas(df):
    df['Frases_normalizadas'] = df['Frases'].apply(lambda x: ' '.join(sorted(x.split())))
    df_filtrado = df.drop_duplicates(subset='Frases_normalizadas')
    return df_filtrado[['Frases']]

frases = [
    #insert here your examples

]
df = criar_dataframe(frases)

df_filtrado = filtrar_frases_repetidas(df)

df_filtrado

,Frases
0,"Gostaria de falar com De repente um atendente,..."
1,"Gostaria falar com um atendente, por favor."
2,"Govtaria de faler com um atendente, por favor."
3,"Gostaria de pronunciar com um atendente,por ge..."
4,"falar com um atendente, por favor. com Gostaria"
...,...
848,Pode me conectar com alguém do suporte at cjie...
849,Pode me ligar com ser humano do apoio ao fregu...
850,alguém do suporte ao cliente? com Pode me cone...
852,conectar me Pode com alguém do suporte ao clie...


In [30]:
def salvar_txt(df_filtrado, filename="frases_filtradas.txt"):
    with open(filename, 'w') as f:
        for frase in df_filtrado['Frases']:
            f.write(f"{frase}\n")

# Exemplo de uso
salvar_txt(df_filtrado, "frases_filtradas.txt")